In [1]:
### pip install and importing packages ###
!sudo apt-get install python3.7
!sudo apt-get update -y
!sudo update-alternatives --install /usr/bin/python3 python3
!sudo update-alternatives --config python3

! pip install recourse==1.0.0
! pip install lime==0.2.0.1

!pip install wandb==0.13.4
!pip install nvidia-ml-py3

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CurrentWork/CARLA - PPM

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support
  python3.7-minimal
Suggested packages:
  python3.7-venv binfmt-support
The following NEW packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support python3.7
  python3.7-minimal
0 upgraded, 6 newly installed, 0 to remove and 35 not upgraded.
Need to get 4,698 kB of archives.
After this operation, 17.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.7-minimal amd64 3.7.17-1+jammy1 [608 kB]
Get:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-minima

In [ ]:
import sys
import os
# set logging
import logging
logging.getLogger().setLevel(logging.INFO)
sys.path.append(os.getcwd())
import random
import numpy as np
import pandas as pd
# packages from https://github.com/irhete/predictive-monitoring-benchmark/blob/master/experiments/experiments.py
from util.DatasetManager import DatasetManager
from util.settings import global_setting, model_setting, training_setting
from carla.models.catalog.LSTM_TORCH.model_lstm import LSTMModel
#user-specified packages
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import roc_auc_score
# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
# to explicitly raise an error with a stack trace to easier debug which operation might have created the invalid values
torch.autograd.set_detect_anomaly(True)
#hyperopt
from hyperopt import hp, Trials, fmin, tpe, STATUS_OK
from hyperopt.pyll.base import scope
from carla.data.catalog.own_catalog import OwnCatalog
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, TensorDataset
from carla.models.catalog.LSTM_TORCH import LSTMModel, CheckpointSaver
import sklearn

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)
##################################################################

def create_and_evaluate_model(args):
    global trial_nr
    trial_nr += 1
    train_cv = torch.zeros((data_cv[0].shape[0], data_cv[0].shape[1], data_cv[0].shape[2]))
    train_y = np.zeros((data_cv[0].shape[0]))
    for cv_iter in range(n_splits):
        val_cv = data_cv[cv_iter]
        val_y = target_cv[cv_iter]
        for cv_train_iter in range(n_splits):
            if cv_train_iter != cv_iter:
                train_cv = torch.cat((train_cv, data_cv[cv_train_iter]), dim=0)
                train_y = np.concatenate((train_y, target_cv[cv_train_iter]), axis=0)

        preds_all = []
        val_y_all = []
        score = 0

        dropout = args['dropout']
        hidden_dim1 = args['hidden_dim1']
        hidden_dim2 = args['hidden_dim2']
        lstm_size = args['lstm_size']
        learning_rate = args['learning_rate']
        clip = training_setting["clip"]

        train_cv = torch.LongTensor(train_cv.long()).to(device)
        train_y = torch.LongTensor(train_y).to(device)
        train_dataset = TensorDataset(train_cv, train_y)
        train_loader = DataLoader(train_dataset, batch_size= 128, shuffle=True, drop_last=True, worker_init_fn=seed_worker)

        model = LSTMModel(dataset.vocab_size, dropout, [hidden_dim1, hidden_dim2, lstm_size])

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas = (0.9, 0.999), amsgrad=False)

        lr_reducer = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=False,
                                threshold=0.0001, cooldown=0, min_lr=0)
        # checkpoint saver
        checkpoint_saver = CheckpointSaver(dirpath=dir_path, decreasing=True, top_n=1)

        criterion = nn.BCELoss()
        # Define the early stopping patience
        print('training')
        early_start_patience = 15
        early_stop_patience = 60
        best_loss = 9999
        for epoch in range(epochs):
                print("Epoch: ", epoch)
                for i, (data_act, train_batch) in enumerate(train_loader, 0): # loop over the data, and jump with step = bptt.
                    model.train()
                    data_act = data_act.to(device)
                    y_ = model(data_act).to(device)
                    train_batch = train_batch.float().unsqueeze(-1).to(device)
                    train_loss = criterion(y_,train_batch).to(device)
                    train_loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
                    optimizer.step()
                    optimizer.zero_grad()
                with torch.no_grad():
                    model.eval()
                    print('validating')
                    pred = model(val_cv)
                    validation_loss = criterion(pred, torch.tensor(val_y , dtype= torch.float32).unsqueeze(-1).to(device))
                    validation_loss = validation_loss.to('cpu').detach().numpy()
                    lr_reducer.step(validation_loss)
                    # Log evaluation metrics to WandB
                    print('validation_loss',validation_loss)
                    print(f"val AUC: {sklearn.metrics.roc_auc_score(val_y, pred.detach().numpy())}")
                    if epoch > early_start_patience:
                        checkpoint_saver(model, epoch, validation_loss, learning_rate, dropout, hidden_dim1, hidden_dim2, lstm_size)
                    if validation_loss < best_loss:
                        best_loss = validation_loss
                if epoch > early_stop_patience and validation_loss > best_loss:
                    print('best loss', best_loss)
                    print('best val loss', validation_loss)
                    print(f"best val AUC: {sklearn.metrics.roc_auc_score(val_y, pred.detach().numpy())}")
                    print("Early stopping triggered.")
                    break
                val_y_all.extend(val_y)
                preds_all.extend(pred)
        score += roc_auc_score(val_y_all, preds_all)
    for k, v in args.items():
        fout_all.write("%s;%s;%s;%s;%s;%s\n" % (trial_nr, dataset_name, cls_method, k, v, score / n_splits))

    fout_all.write("%s;%s;%s;%s\n" % (trial_nr, dataset_name, cls_method, 0))
    fout_all.flush()
    return {'loss': validation_loss,
            'status': STATUS_OK,
            'model': model,
            'args': args}

dataset_names = ['production','sepsis_cases_1', 'sepsis_cases_2', 'sepsis_cases_4', 'bpic2012_accepted', 'bpic2012_rejected', 'bpic2012_cancelled']
dataset_name = 'bpic2012_accepted'

print('own dataset', dataset_name)
dataset = OwnCatalog(dataset_name)

dataset_manager = DatasetManager(dataset_name)

cls_method = 'LSTM'
path = global_setting['params_dir_LSTM']
dir_path = global_setting['hyper_models'] +dataset_name
dataset_manager.ensure_path(dir_path)

seed = global_setting['seed']
epochs = training_setting['epochs_LSTM']
n_splits = global_setting['n_splits']
max_evals = global_setting['max_evals']
train_ratio = global_setting['train_ratio']

print('Dataset:', dataset_name)

# we only use the training dataset for hyperoptimization
data_train = dataset.df_train
target_train = dataset.target_train

skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(data_train, target_train)

data_cv = []
target_cv = []
for i, (train_index, val_index) in enumerate(skf.split(data_train, target_train)):
    data_cv.append(data_train[val_index]) # you add all the val chunks
    target_cv.append(target_train[val_index])

# set up search space
space = {   'dropout'      : hp.choice('dropout', [0.0, 0.1, 0.2, 0.3]),
            'hidden_dim1'  : scope.int(hp.quniform('hidden_dim1',50,100,50)),
            'hidden_dim2'  : scope.int(hp.quniform('hidden_dim2',10,50,10)),
            'lstm_size'    : scope.int(hp.quniform('lstm_size',2,5,1)),
            'learning_rate': hp.choice('learning_rate', [0.001, 0.002,0.005, 0.01]),
                 }

# optimize parameters
trial_nr = 1 # initialize the trail number
trials = Trials()
fout_all = open(os.path.join(path, "param_optim_all_trials_%s_%s.csv" % (cls_method, dataset_name)), "w")
rstate = random.seed(22) # Set the seed
trials = Trials()
best = fmin(create_and_evaluate_model, space, algo=tpe.suggest, max_evals= global_setting['max_evals'], trials=trials, rstate = rstate)
fout_all.close()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
own dataset bpic2012_accepted


/content/drive/MyDrive/CurrentWork/CARLA - PPM/util/DatasetManager.py:60: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  loaded_data = torch.LongTensor(hdf5_file['data'])


Streaming output truncated to the last 5000 lines.
validation_loss
0.27806967
val AUC: 0.9453752181500872
Epoch: 
40
validating
validation_loss
0.28510767
val AUC: 0.9450451181730861
Epoch: 
41
validating
validation_loss
0.27694157
val AUC: 0.9459055427033024
Epoch: 
42
validating
validation_loss
0.27279148
val AUC: 0.9466780307642355
 47%|████▋     | 7/15 [6:14:33<7:09:41, 3222.71s/trial, best loss: 0.25144919753074646][INFO] Current metric value better than 0.27279147505760193 better than best 0.2760101854801178, saving model at carla/hyperoptimization/params_dir_LSTM/hyper_models/bpic2012_accepted/LSTMModel_0.001_50_40_4_0.0_epoch42.pt [model_lstm.py __call__]
[INFO] Removing extra models.. [{'path': 'carla/hyperoptimization/params_dir_LSTM/hyper_models/bpic2012_accepted/LSTMModel_0.001_50_40_4_0.0_epoch36.pt', 'score': array(0.2760102, dtype=float32)}] [model_lstm.py cleanup]
Epoch: 
43
validating
validation_loss
0.27975166
val AUC: 0.9458311349216011
Epoch: 
44
validating
validati

In [ ]:
print('hello')